In [ ]:
from section3_1 import get_A_HCZ_i

## 付録H　温水暖房用熱源機の最大能力

### H.1 記号及び単位

省略

### H.2 温水暖房用熱源機の最大能力

$$\large
q_{max,hs} = q_{rq,H} \times \sum_{i} A_{HCZ,i} \times f_{cT} \times f_{cI}
$$

$q_{max,hs}$：温水暖房用熱源機の最大能力(W)  
$q_{rq,H}$：単位面積当たりの必要暖房能力(W/m<sup>2</sup>)  
$A_{HCZ,i}$：暖冷房区画$i$の床面積(m<sup>2</sup>)  
$f_{cT}$：外気温度能力補正係数  
$f_{cI}$：間歇運転能力補正係数

In [ ]:
# 温水暖房用熱源機の最大能力 (1)
def get_q_max_hs(region, A_A, A_MR, A_OR, mode_MR, mode_OR, has_MR_hwh, has_OR_hwh):
    """
    :param region: 省エネルギー地域区分
    :param A_A: 床面積の合計 (m2)
    :param A_MR: 主たる居室の床面積 (m2)
    :param A_OR: その他の居室の床面積 (m2)
    :param has_MR_hwh: 温水暖房の放熱器を主たる居室に設置する場合はTrue
    :param has_OR_hwh: 温水暖房の放熱器をその他の居室に設置する場合はTrue
    :param has_MR_hwh: 温水暖房の放熱器を主たる居室に設置する場合はTrue
    :param has_OR_hwh: 温水暖房の放熱器をその他の居室に設置する場合はTrue
    :return: 温水暖房用熱源機の最大能力 (W)
    """
    # 単位面積当たりの必要暖房能力
    q_rq_H = get_q_rq_H(region, has_MR_hwh, has_OR_hwh)

    # 外気温度補正係数
    f_cT = get_f_cT()

    # 間歇運転能力補正係数
    f_cI = get_f_cI(mode_MR, mode_OR, has_MR_hwh, has_OR_hwh)

    # 暖冷房区画の床面積(温水暖房により暖 房される暖冷房区画のみを積算する)
    A_HCZ_hs = 0
    if has_MR_hwh:
        A_HCZ_hs = get_A_HCZ_i(1, A_A, A_MR, A_OR)
    if has_OR_hwh:
        A_HCZ_hs = A_HCZ_hs + sum([get_A_HCZ_i(i, A_A, A_MR, A_OR) for i in range(2, 6)])

    return q_rq_H * A_HCZ_hs * f_cT * f_cI


# 単位面積当たりの必要暖房能力
def get_q_rq_H(region, has_MR_hwh, has_OR_hwh):
    """
    :param region: 省エネルギー地域区分
    :param has_MR_hwh: 温水暖房の放熱器を主たる居室に設置する場合はTrue
    :param has_OR_hwh: 温水暖房の放熱器をその他の居室に設置する場合はTrue
    :return: 単位面積当たりの必要暖房能力 (W/m2)
    """
    if has_MR_hwh and has_OR_hwh:
        return table_h_3[region - 1][0]
    elif has_MR_hwh:
        return table_h_3[region - 1][1]
    elif has_OR_hwh:
        return table_h_3[region - 1][2]
    else:
        raise ValueError('温水暖房の放熱器を主たる居室にもその他の居室にも設置しない場合の単位面積当たりの必要暖房能力は定義されません。')


外気温度補正係数$f_{cT}$は、1.05の値とする。

In [ ]:
# 外気温度補正係数
def get_f_cT():
    return 1.05

In [ ]:
# 間歇運転能力補正係数
def get_f_cI(mode_MR, mode_OR, has_MR_hwh, has_OR_hwh):
    """
    :param mode_MR: 主たる居室の運転方法 (連続運転|間歇運転)
    :param mode_OR: その他の居室の運転方法 (連続運転|間歇運転)
    :param has_MR_hwh: 温水暖房の放熱器を主たる居室に設置する場合はTrue
    :param has_OR_hwh: 温水暖房の放熱器をその他の居室に設置する場合はTrue
    :return: 間歇運転能力補正係数
    """

    def normalize_mode(s):
        if s == 'は':
            return '間歇運転'
        if s == 'ろ':
            return '連続運転'
        return s

    mode_MR = normalize_mode(mode_MR)
    mode_OR = normalize_mode(mode_OR)

    if has_MR_hwh and has_OR_hwh:
        if mode_MR == '連続運転':
            if mode_OR == '連続運転':
                return table_h_4[0]
            elif mode_OR == '間歇運転':
                return table_h_4[1]
            else:
                raise ValueError(mode_OR)
        elif mode_MR == '間歇運転':
            if mode_OR == '連続運転':
                return table_h_4[2]
            elif mode_OR == '間歇運転':
                return table_h_4[3]
            else:
                raise ValueError(mode_MR)
        else:
            raise ValueError(mode_MR)
    elif has_MR_hwh:
        if mode_MR == '連続運転':
            return table_h_4[4]
        elif mode_MR == '間歇運転':
            return table_h_4[5]
        else:
            raise ValueError(mode_MR)
    elif has_OR_hwh:
        if mode_OR == '連続運転':
            return table_h_4[6]
        elif mode_OR == '間歇運転':
            return table_h_4[7]
        else:
            raise ValueError(mode_OR)

<div style="text-align: center;font-weight: bold;">表　単位面積当たりの必要暖房能力(W/m<sup>2</sup>)</div>

|地域の区分|（い）温水暖房の放熱器を主たる居室及びその他の居室に設置する場合|（ろ）温水暖房の放熱器を主たる居室に設置しその他の居室に設置しない場合|（は）温水暖房の放熱器を主たる居室に設置せずその他の居室に設置する場合|
|-|-|-|-|
|1|90.02|139.26|62.28|
|2|77.81|120.65|53.26|
|3|73.86|111.32|53.81|
|4|77.74|118.98|55.41|
|5|83.24|126.56|59.43|
|6|69.76|106.48|49.93|
|7|74.66|112.91|53.48|
|8|－|－|－|


In [ ]:
# 表 H.3 単位面積当たりの必要暖房能力
table_h_3 = [
    (90.02, 139.26, 62.28),
    (77.81, 120.65, 53.26),
    (73.86, 111.32, 53.81),
    (77.74, 118.98, 55.41),
    (83.24, 126.56, 59.43),
    (69.76, 106.48, 49.94),
    (74.66, 112.91, 53.48)
]

<div style="text-align: center;font-weight: bold;">表　間歇運転能力補正係数</div>

|主たる居室の運転方法|その他の居室の運転方法|間歇運転能力補正係数|
|-|-|-|
|連続運転|連続運転|1.0|
|連続運転|間歇運転|1.0|
|間歇運転|連続運転|2.25|
|間歇運転|間歇運転|2.25|
|連続運転|放熱器を設置しない<sup>※</sup>|1.0|
|間歇運転|放熱器を設置しない<sup>※</sup>|3.03|
|放熱器を設置しない|連続運転|1.0|
|放熱器を設置しない|間歇運転|1.62|


※ その他の居室が無い場合も含む。 

In [1]:
# 表 H.4 間歇運転能力補正係数
table_h_4 = [
    1.0,
    1.0,
    2.25,
    2.25,
    1.0,
    3.03,
    1.0,
    1.62
]